# [실습] 아이템 기반 최근접 이웃 협업 필터링 (knn)
- 최근접 이웃 협업 필터링은 사용자기반과 아이템기반으로 분류함, 일반적으로 아이템기반 협업 필터링을 많이 사용

In [37]:
import pandas as pd
import numpy as np

In [38]:
movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
print(movies.shape); print(ratings.shape)

(9742, 3)
(100836, 4)


In [39]:
print(movies.columns)
print(ratings.columns)

Index(['movieId', 'title', 'genres'], dtype='object')
Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


In [40]:
ratings = ratings[['userId', 'movieId', 'rating']]
rating_movies = pd.merge(ratings, movies, on='movieId')


ratings_matrix = rating_movies.pivot_table('rating', index='userId',columns='title')
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1. 영화간 유사도 산출
- cosine similarity()는 기준행과 타행을 비교하여 유사도 산출 

In [41]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T,ratings_matrix_T)
item_sim_df = pd.DataFrame(item_sim,index=ratings_matrix.columns, columns=ratings_matrix.columns)

In [43]:
item_sim_df.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Salem's Lot (2004),0.0,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Til There Was You (1997),0.0,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


## 2 아이템기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천
- 개인화된 영화 추천은 개인이 아직 관람하지 않은 영화를 추천한다는 것임. 아직 관람하지 않은 영화 평점에 대해 아이템 유사도와 기존에 관람한 영화의 평점 데이터를 기반으로 해 새롭게 모든영화의 예측 평점을 계산한 후 높은 예측 평점을 가진 영화를 추천하는 방식.

In [48]:
def predict_rating(ratings_arr, item_sim_arr) :
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

- 예측평점이 사용자별 영화의 실제 평점과 영화의 코사인 유사도를 내적(dot)한 값이므로 기존에 0이 였던 평점이 다른 값이 부여되는 경우가 발생함
- 예측평점이 실제 평점에 비해 작을 수 있음. (코사인 유사도 벡터합으로 나누었기에 생기는 현상)

In [52]:
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)

ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501


### 2.1 예측결과와 실제 평점이 얼마나 차이가 있는지 확인

In [53]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE를 구함
def get_mse(pred, actual):
    # 평점이 있는 실제 영화만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred,actual)

print('아이템 기반 모든 최근접 이웃 MSE:', get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE: 9.895354759094706


### 2.2 MSE를 감소시켜보자.
- 예측평점을 계산할때 모든 영화와 비교를 진행했음. 
- 매우 많은 영화를 이용하다 보나 상대적으로 평점 예측이 떨어짐. 
- 특정영화와 가장 비슷한 유사도를 가지는 영화에서만 유사도를 적용하는 것으로 바꿔보자

In [93]:
# 아쉬운 점  : 개별  예측값을 구하기 위해서 행/열 별로 for문 수행 -> 데이터 크기가 커질수록 매우 오래 걸리는 로직

def predict_rating_topsim(rating_arr, item_sim_arr, n=20) :
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측행렬 초기화
    pred = np.zeros(rating_arr.shape)
    
    # 사용자-아이템 평점행렬의 열 크기 만큼 확인
    for col in range(rating_arr.shape[1]):
        # 유사도 행열에서 유사도가 큰 순으로 n개 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[:,col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(rating_arr.shape[0]):
            pred[row,col] = item_sim_arr[col,:][top_n_items].dot(rating_arr[row,:][top_n_items].T)
            pred[row,col] /= np.sum(np.abs(item_sim_arr[col,:][top_n_items]))
    return pred

In [94]:
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)
print('아이템기반 최근점 TOP-20 이웃 MSE:', get_mse(ratings_pred, ratings_matrix.values))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns = ratings_matrix.columns)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


아이템기반 최근점 TOP-20 이웃 MSE: 3.694999233129397


### 2.3 사용자가 이미 평점을 준 영화를 제외하고 추천 할 수 있도록 해보자

In [106]:
# 9번 사용자가 준 평점이 높은 순으로 나열 해보자 
user_rating_id = ratings_matrix.loc[9,:]
user_rating_id[user_rating_id>0].sort_values(ascending=False)[:10]

title
Adaptation (2002)                                                                 5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Producers, The (1968)                                                             5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Back to the Future (1985)                                                         5.0
Austin Powers in Goldmember (2002)                                                5.0
Minority Report (2002)                                                            4.0
Witness (1985)                                                                    4.0
Name: 9, dtype: float64

In [126]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series로 반환
    # 반환된 user_rating은 영화명을 인덱스로 가지는 Series 객체임
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상인덱스를 추출해 list 객체로 만듦
    idx = user_rating > 0
    already_seen =user_rating[idx].index.tolist()
    
    # 모든 영화명을 list 객체로 만듦
    movies_list =ratings_matrix.columns.tolist()
    
    #list comprehension으로 already_seen에 해당하는 영화는 movies_list에서 제외함.
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

- 사용자가 영화의 평점을 주지 않은 추천 대상 영화 정보와 사용자별 아이템 유사도에 기반한 예측 평점 데이터 세트를 이용해 최종적으로 사용자에게 영화를 추천하는 함수를 만들어보자.

In [111]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자 id 인덱스와 unseen_list로 돌아온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬
    recomm_movies = pred_df.loc[userId,unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [132]:
# 9번 사용자가 관람하지 않는 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix,9)

# 아이템 기반의 최근접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9 ,unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로 생성.
recomm_movies= pd.DataFrame(data=recomm_movies.values, index = recomm_movies.index, columns=['pred_score'])
recomm_movies


,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


# FinalPJT 데이터를 활용해보자.

In [1]:
import pandas as pd
import numpy as np

In [2]:
place = pd.read_csv('./wwg_place.csv')

In [3]:
place.head()

,번호,상호명,전화번호,소재지,업종,설명(판매메뉴),imgURl,jpg,위도,경도
0,1.0,본죽&비빔밥cafe (서초법원),02-525-2909,서울특별시 서초구 반포대로30길 53,한식,특전복내장죽(페스코) 특전복죽(페스코) 진전복죽(페스코) 전복죽(페스코) 매생이굴죽...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",본죽&비빔밥cafe (서초법원).jgp,37.492179,127.011001
1,2.0,본죽&비빔밥cafe (반포서래),02-537-6288,서울특별시 서초구 서래로 31,한식,특전복내장죽(페스코) 특전복죽(페스코) 진전복죽(페스코) 전복죽(페스코) 매생이굴죽...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",본죽&비빔밥cafe (반포서래).jgp,37.497390,126.998310
2,3.0,본죽&비빔밥cafe (명동2호),02-778-3562,서울특별시 중구 명동9길 10,한식,특전복내장죽(페스코) 특전복죽(페스코) 진전복죽(페스코) 전복죽(페스코) 매생이굴죽...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",본죽&비빔밥cafe (명동2호).jgp,37.564980,126.984467
3,4.0,본죽&비빔밥cafe (둔촌보훈병원),02-478-6288,서울특별시 강동구 진황도로 189,한식,특전복내장죽(페스코) 특전복죽(페스코) 진전복죽(페스코) 전복죽(페스코) 매생이굴죽...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",본죽&비빔밥cafe (둔촌보훈병원).jgp,37.534550,127.139775
4,5.0,본죽&비빔밥cafe (돈암동),02-928-8477,서울특별시 성북구 아리랑로 6-1,한식,특전복내장죽(페스코) 특전복죽(페스코) 진전복죽(페스코) 전복죽(페스코) 매생이굴죽...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",본죽&비빔밥cafe (돈암동).jgp,37.602664,127.013449
